In [1]:
import qutip as qt
import numpy as np
import scipy as sp

In [2]:
sx = 0.5*qt.operators.sigmax()
sz = 0.5*qt.operators.sigmaz()
I = qt.operators.identity(2)
sm = qt.operators.sigmam()

In [22]:
def hamiltonian_two_level(N,omega,delta):
  sx = 0.5*qt.operators.sigmax()
  sz = 0.5*qt.operators.sigmaz()
  I = qt.operators.identity(2)

  ham = qt.tensor([I*0]*N)
  for i in range(N):
    hz = qt.tensor([sz if j==i else I for j in range(N)])
    hx = qt.tensor([sx if j==i else I for j in range(N)])
    ham += delta*hz+omega*hx
    
  
  return(ham)

In [23]:
N = 1
omega = 1
delta = -1

ham = hamiltonian_two_level(N,omega,delta)
ham

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[-0.5  0.5]
 [ 0.5  0.5]]

In [24]:
I = qt.operators.identity(2)
Id = qt.tensor([I]*N)

In [36]:
coherent = -1j*(qt.tensor(Id,ham)-qt.tensor(ham.trans(),Id))
coherent

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.+0.j  0.-0.5j 0.+0.5j 0.+0.j ]
 [0.-0.5j 0.-1.j  0.+0.j  0.+0.5j]
 [0.+0.5j 0.+0.j  0.+1.j  0.-0.5j]
 [0.+0.j  0.+0.5j 0.-0.5j 0.+0.j ]]

In [38]:
kappa = 1
c_ops = [qt.tensor([sm if i==j  else I for j in range(N)]) for i in range(N)]
c_ops = [np.sqrt(kappa) * c_op for c_op in c_ops]
c_tops = [c.dag() for c in c_ops]

In [39]:
dissipative_part = sum([qt.tensor(Ad.trans(),A)-0.5*qt.tensor(Id,Ad*A)-0.5*qt.tensor((Ad*A).trans(),Id) for Ad,A in zip(c_tops,c_ops)])
dissipative_part

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-1.   0.   0.   0. ]
 [ 0.  -0.5  0.   0. ]
 [ 0.   0.  -0.5  0. ]
 [ 1.   0.   0.   0. ]]

In [40]:
Linbladian = coherent+dissipative_part
Linbladian

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-1. +0.j   0. -0.5j  0. +0.5j  0. +0.j ]
 [ 0. -0.5j -0.5-1.j   0. +0.j   0. +0.5j]
 [ 0. +0.5j  0. +0.j  -0.5+1.j   0. -0.5j]
 [ 1. +0.j   0. +0.5j  0. -0.5j  0. +0.j ]]

In [41]:
val,mat = np.linalg.eig(Linbladian)

In [42]:
val

array([-6.20972064e-01-1.38669775e+00j, -6.20972064e-01+1.38669775e+00j,
        2.73135115e-16+1.47924831e-17j, -7.58055872e-01-1.86951828e-16j])

In [43]:
np.matrix(mat[2].reshape(2,2))

matrix([[-0.14092519+0.03636657j,  0.85841332+0.j        ],
        [ 0.29172998-0.14586499j, -0.47627547-0.12290568j]])